In [1]:
# Gerekli kütüphaneler
import pandas as pd
import numpy as np

In [2]:
CSV_PATH = "match_odds_wide.csv"
KEY_COLS = [
    "match_date", "match_time", "tournament", "match_id",
    "homeTeam", "awayTeam",
    "firstHalfHomeGoal", "firstHalfAwayGoal",
    "totalHomeGoal", "totalAwayGoal",
    "homeCorner", "awayCorner"
]

In [3]:
df = pd.read_csv(CSV_PATH)

In [4]:
numeric_cols = [col for col in df.columns if col not in KEY_COLS]

In [5]:
train_set = df.dropna(subset=["totalHomeGoal", "totalAwayGoal"])
train_set.shape

(2470, 780)

In [6]:
test_set = df[df[["totalHomeGoal", "totalAwayGoal"]].isna().any(axis=1)]
test_set.shape

(754, 780)

In [7]:
# 3. Özellik sütunlarını belirle
feature_cols = [col for col in df.columns if col not in KEY_COLS]

# Sayısal verileri al (kategorik varsa elleme hatası olur)
feature_cols = [col for col in feature_cols if pd.api.types.is_numeric_dtype(df[col])]


In [8]:
# Train: skoru olanlar, Test: skoru olmayanlar
train_df = df[df["totalHomeGoal"].notna()].copy()
test_df  = df[df["totalHomeGoal"].isna()].copy()

# Sadece oran sütunları
ODDS_COLS = [col for col in df.columns if col not in KEY_COLS]

train_odds = train_df[ODDS_COLS]
test_odds = test_df[ODDS_COLS]

In [9]:
train_odds

,1.Gol Hangi Dakika Aralığında Olur :: 1-10 dk.,1.Gol Hangi Dakika Aralığında Olur :: 11-20 dk.,1.Gol Hangi Dakika Aralığında Olur :: 21-30 dk.,1.Gol Hangi Dakika Aralığında Olur :: 31-40 dk.,1.Gol Hangi Dakika Aralığında Olur :: 41-50 dk.,1.Gol Hangi Dakika Aralığında Olur :: 51-60 dk.,1.Gol Hangi Dakika Aralığında Olur :: 61-70 dk.,1.Gol Hangi Dakika Aralığında Olur :: 71-80 dk.,1.Gol Hangi Dakika Aralığında Olur :: 81-90 dk.,1.Gol Hangi Dakika Aralığında Olur :: Olmaz,...,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Var,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Yok,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Var,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Yok,İlk Yarı Çifte Şans :: İY 1-2,İlk Yarı Çifte Şans :: İY 1-X,İlk Yarı Çifte Şans :: İY X-2,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Dep,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Ev,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Eşit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.00,3.58,5.84,2.85,1.34,1.12,1.25,NaN,NaN,NaN
2625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.00,4.29,10.10,1.72,1.65,1.00,1.14,NaN,NaN,NaN
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.45,2.50,6.60,2.52,1.38,1.27,1.07,NaN,NaN,NaN
2627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.68,2.09,6.14,3.12,1.28,1.44,1.00,NaN,NaN,NaN


In [10]:
# -- 2. Testten rastgele bir maç seç
import random

random_idx = random.randint(0, len(test_df)-1)
test_row = test_df.iloc[random_idx]

In [11]:
test_row["Maç Sonucu :: MS 2"]

7.97

In [12]:
test_row = test_df[test_df["match_id"] == 2154719].copy()
test_row

,match_date,match_time,tournament,match_id,homeTeam,awayTeam,firstHalfHomeGoal,firstHalfAwayGoal,totalHomeGoal,totalAwayGoal,...,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Var,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY 2 ve Yok,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Var,İlk Yarı Sonucu ve İY Karşılıklı Gol :: İY X ve Yok,İlk Yarı Çifte Şans :: İY 1-2,İlk Yarı Çifte Şans :: İY 1-X,İlk Yarı Çifte Şans :: İY X-2,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Dep,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Ev,İlk Yarıda Hangi Takım Daha Fazla Korner Kullanır :: Eşit


In [13]:
train_df = df[df["totalHomeGoal"].notna()].reset_index(drop=True)
test_df = df[df["totalHomeGoal"].isna()].reset_index(drop=True)
# -- 3. En yakın 100 maçı bul
distances = []

for idx, train_row in train_df.iterrows():
    train_vals = train_row[numeric_cols]
    test_vals = test_row[numeric_cols]
    
    test_vals = test_vals.iloc[0]

    valid_mask = ~(train_vals.isna() | test_vals.isna())

    if valid_mask.sum() == 0:
        dist = np.inf
    else:
        diff_squared = (train_vals[valid_mask] - test_vals[valid_mask]) ** 2
        dist = np.sqrt(diff_squared.sum())

    distances.append((idx, dist))

distance_df = pd.DataFrame(distances, columns=["index", "distance"])
top_100_indices = distance_df.sort_values(by="distance").head(43)

# ✅ Fixed indexing issue here
top_100_matches = train_df.loc[top_100_indices["index"]]

IndexError: single positional indexer is out-of-bounds

In [14]:
import pandas as pd
import numpy as np
import random

CSV_PATH = "match_odds_wide.csv"

# -- 1. Veri okuma ve ayrıştırma
df = pd.read_csv(CSV_PATH)

df = df.sample(frac=1)


meta_cols = [
    'match_date', 'match_time', 'tournament', 'match_id',
    'homeTeam', 'awayTeam',
    'firstHalfHomeGoal', 'firstHalfAwayGoal',
    'totalHomeGoal', 'totalAwayGoal',
    'homeCorner', 'awayCorner'
]

numeric_cols = [col for col in df.columns if col not in meta_cols]

train_df = df[df["totalHomeGoal"].notna()].reset_index(drop=True)
test_df = df[df["totalHomeGoal"].isna()].reset_index(drop=True)

# -- 2. Testten rastgele bir maç seç
random_idx = random.randint(0, len(test_df) - 1)
test_row = test_df.iloc[random_idx]

test_df = test_df.sample(frac=1)


for t_id, t_row in test_df.iterrows():
    test_row = test_df.iloc[t_id]


    # -- 3. En yakın 100 maçı bul
    distances = []

    for idx, train_row in train_df.iterrows():
        train_vals = train_row[numeric_cols]
        test_vals = test_row[numeric_cols]

        valid_mask = ~(train_vals.isna() | test_vals.isna())

        if valid_mask.sum() == 0:
            dist = np.inf
        else:
            diff_squared = (train_vals[valid_mask] - test_vals[valid_mask]) ** 2
            dist = np.sqrt(diff_squared.sum())

        distances.append((idx, dist))

    distance_df = pd.DataFrame(distances, columns=["index", "distance"])
    top_100_indices = distance_df.sort_values(by="distance").head(43)

    # ✅ Fixed indexing issue here
    top_100_matches = train_df.loc[top_100_indices["index"]]

    # -- 4. MS 1-X-2 sonuçlarını hesapla
    ms1 = ((top_100_matches["totalHomeGoal"] > top_100_matches["totalAwayGoal"])).sum()
    msx = ((top_100_matches["totalHomeGoal"] == top_100_matches["totalAwayGoal"])).sum()
    ms2 = ((top_100_matches["totalHomeGoal"] < top_100_matches["totalAwayGoal"])).sum()

    total = ms1 + msx + ms2

    ms1_prob = ms1 / total
    msx_prob = msx / total
    ms2_prob = ms2 / total



    # # -- 5. Sonuçları yazdır
    # print(f"Test Maçı: {test_row['homeTeam']} vs {test_row['awayTeam']}")
    # print(f"\nTop 100 en yakın maç bazında MS olasılıkları:")
    # print(f"  MS 1: %{ms1_prob * 100:.2f}")
    # print(f"  MS X: %{msx_prob * 100:.2f}")
    # print(f"  MS 2: %{ms2_prob * 100:.2f}")
    # print(test_row["Maç Sonucu :: MS 1"])
    # print(test_row["Maç Sonucu :: MS X"])
    # print(test_row["Maç Sonucu :: MS 2"])
    # print(f"  MS 1: %{test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100:.2f}")
    # print(f"  MS X: %{test_row["Maç Sonucu :: MS X"]*msx_prob * 100:.2f}")
    # print(f"  MS 2: %{test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100:.2f}")

    values = [test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100,
            test_row["Maç Sonucu :: MS X"]*msx_prob * 100,
            test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100
            ]
    if max(values) >= 100:
        print(f"Test Maçı: {test_row['homeTeam']} vs {test_row['awayTeam']}")
        print(f"\nTop 100 en yakın maç bazında MS olasılıkları:")
        print(f"  MS 1: %{ms1_prob * 100:.2f}")
        print(f"  MS X: %{msx_prob * 100:.2f}")
        print(f"  MS 2: %{ms2_prob * 100:.2f}")
        print(test_row["Maç Sonucu :: MS 1"])
        print(test_row["Maç Sonucu :: MS X"])
        print(test_row["Maç Sonucu :: MS 2"])
        print(f"  MS 1: %{test_row["Maç Sonucu :: MS 1"]*ms1_prob * 100:.2f}")
        print(f"  MS X: %{test_row["Maç Sonucu :: MS X"]*msx_prob * 100:.2f}")
        print(f"  MS 2: %{test_row["Maç Sonucu :: MS 2"]*ms2_prob * 100:.2f}")
        print(values)
        print("-"*50)


Test Maçı: Sola vs Torvastad

Top 100 en yakın maç bazında MS olasılıkları:
  MS 1: %55.81
  MS X: %9.30
  MS 2: %34.88
1.9
3.61
2.42
  MS 1: %106.05
  MS X: %33.58
  MS 2: %84.42
[106.04651162790697, 33.581395348837205, 84.41860465116278]
--------------------------------------------------
Test Maçı: FC Irp Cesky Tesin vs Trinec

Top 100 en yakın maç bazında MS olasılıkları:
  MS 1: %13.95
  MS X: %23.26
  MS 2: %62.79
8.27
6.4
1.06
  MS 1: %115.40
  MS X: %148.84
  MS 2: %66.56
[115.3953488372093, 148.8372093023256, 66.55813953488372]
--------------------------------------------------
Test Maçı: Finn Harps vs Cobh Rambler

Top 100 en yakın maç bazında MS olasılıkları:
  MS 1: %16.28
  MS X: %32.56
  MS 2: %51.16
3.54
3.08
1.65
  MS 1: %57.63
  MS X: %100.28
  MS 2: %84.42
[57.62790697674419, 100.27906976744188, 84.41860465116278]
--------------------------------------------------
Test Maçı: Tatran Vsechovice vs TJ Unie Hlubina

Top 100 en yakın maç bazında MS olasılıkları:
  MS 1: %27

KeyboardInterrupt: 